# Init

In [1]:
%%capture
import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
import re
from unidecode import unidecode

2023-02-03 10:26:40.477354: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-03 10:26:40.897803: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-03 10:26:40.897851: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-03 10:26:42.659541: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-0

In [2]:
data = pd.read_csv("train_ner.csv")
data.head()

,id,text,label
0,32674,da de unde stii u mai [ORG] ca banii au fost p...,ABUSE
1,16514,m*uie [PERS] m*uie [PERS] ... m*uie\nbai kakat...,INSULT
2,32556,PT ALA CARE ARE TREABA CU [PERS]!!ESTI UNUL CA...,OTHER
3,23861,sunt bucuros ca [PERS] nu a mai venit la [ORG]...,OTHER
4,21811,[PERS] esti....PE..N\n ES..CU..LI..BI.L!!! te ...,INSULT


In [3]:
import spacy

2023-02-03 10:26:47.956492: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-03 10:26:47.956907: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-03 10:26:47.956967: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (andrei-VirtualBox): /proc/driver/nvidia/version does not exist


In [4]:
nlp = spacy.load('ro_core_news_lg')


In [5]:
stops = nlp.Defaults.stop_words

In [6]:
def preprocess(x):
    s = unidecode(x)
    s = str.lower(s)
    s = re.sub(r"\[[a-z]+\]","", s)
    s = re.sub(r"\*","", s)
    s = re.sub(r"[^a-zA-Z0-9]+"," ",s)
    s = re.sub(r" +"," ",s)
    s = re.sub(r"(.)\1+",r"\1",s)
    s = " ".join([elem for elem in s.split(' ') if elem not in stops])
    return s

data["preprocessed"] = data["text"].map(preprocess)
data.head(5)

,id,text,label,preprocessed
0,32674,da de unde stii u mai [ORG] ca banii au fost p...,ABUSE,sti bani pt pt dulce scumpa bomboana adik pron...
1,16514,m*uie [PERS] m*uie [PERS] ... m*uie\nbai kakat...,INSULT,muie muie muie bai kakatule stai banca arbitru...
2,32556,PT ALA CARE ARE TREABA CU [PERS]!!ESTI UNUL CA...,OTHER,pt treaba esti ide fotbal
3,23861,sunt bucuros ca [PERS] nu a mai venit la [ORG]...,OTHER,bucuros venit jucator duzina spere echipa nati...
4,21811,[PERS] esti....PE..N\n ES..CU..LI..BI.L!!! te ...,INSULT,esti es bi asemeni chiajna parveniti fotbal g...


In [7]:
label_ids = {label_name:i for i, label_name in enumerate(sorted(set(data["label"])))}
label_ids

{'ABUSE': 0, 'INSULT': 1, 'OTHER': 2, 'PROFANITY': 3}

In [8]:
l = [elem.split(" ") for elem in data["preprocessed"].tolist()]
s = set()
for elem in l:
    s = s|set(elem)
print(len(s))

27240


In [9]:
data["class"] = data["label"].map(lambda x: label_ids[x])
data.head(5)

,id,text,label,preprocessed,class
0,32674,da de unde stii u mai [ORG] ca banii au fost p...,ABUSE,sti bani pt pt dulce scumpa bomboana adik pron...,0
1,16514,m*uie [PERS] m*uie [PERS] ... m*uie\nbai kakat...,INSULT,muie muie muie bai kakatule stai banca arbitru...,1
2,32556,PT ALA CARE ARE TREABA CU [PERS]!!ESTI UNUL CA...,OTHER,pt treaba esti ide fotbal,2
3,23861,sunt bucuros ca [PERS] nu a mai venit la [ORG]...,OTHER,bucuros venit jucator duzina spere echipa nati...,2
4,21811,[PERS] esti....PE..N\n ES..CU..LI..BI.L!!! te ...,INSULT,esti es bi asemeni chiajna parveniti fotbal g...,1


In [10]:
MAX_FEATURES = 10000
SEQUENCE_LENGTH = 120
BATCH_SIZE = 32
EMBEDDING_DIM = 400
EPOCHS = 5

In [11]:
def prepare_dataset(filename):
    df = pd.read_csv(filename)
    df["preprocessed"] = df["text"].map(preprocess)
    df["class"] = df["label"].map(lambda x: label_ids[x])
    res_ds = tf.data.Dataset.from_tensor_slices((df["preprocessed"],df["class"])).shuffle(len(df)).batch(BATCH_SIZE)
    return res_ds

In [12]:
train_ds = prepare_dataset("train_ner.csv")
test_ds = prepare_dataset("test_ner.csv")
val_ds = prepare_dataset("validation_internal_ner.csv")

2023-02-03 10:27:07.629486: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
texts = train_ds.map(lambda x,y:x)
vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=MAX_FEATURES,
    output_mode="int",
    output_sequence_length=SEQUENCE_LENGTH,
    standardize=None,
    split="whitespace",
)
vectorizer.adapt(texts)

In [26]:
input = tf.keras.Input(shape=(),dtype=tf.string)
tokens = vectorizer(input)
embed_layer = tf.keras.layers.Embedding(
    input_dim=MAX_FEATURES,
    output_dim=EMBEDDING_DIM,
    mask_zero=True
)
embeds = embed_layer(tokens)
rnn_layer = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(
        units=50,
        return_sequences=True
    ),
    merge_mode="ave"
)
rnn_layer2 = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(
        units=10
    ),
    merge_mode="ave"
)

h = rnn_layer(embeds)
h = rnn_layer2(h)
fc1 = tf.keras.layers.Dense(10,activation="relu")
fc2 = tf.keras.layers.Dense(len(label_ids))

h = fc1(h)
h = tf.keras.layers.Dropout(0.2)(h)
output = fc2(h)

model = tf.keras.Model(inputs=input,outputs=output)

In [27]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

In [28]:
model.fit(x=train_ds, epochs=5, validation_data=val_ds)

Epoch 1/5
312/312 [==============================] - 107s 295ms/step - loss: 1.1439 - sparse_categorical_accuracy: 0.4981 - val_loss: 0.7554 - val_sparse_categorical_accuracy: 0.7529
Epoch 2/5
312/312 [==============================] - 80s 256ms/step - loss: 0.7333 - sparse_categorical_accuracy: 0.7243 - val_loss: 0.4520 - val_sparse_categorical_accuracy: 0.8780
Epoch 3/5
312/312 [==============================] - 79s 254ms/step - loss: 0.4914 - sparse_categorical_accuracy: 0.8334 - val_loss: 0.2570 - val_sparse_categorical_accuracy: 0.9342
Epoch 4/5
312/312 [==============================] - 78s 251ms/step - loss: 0.3273 - sparse_categorical_accuracy: 0.9018 - val_loss: 0.1414 - val_sparse_categorical_accuracy: 0.9684
Epoch 5/5
312/312 [==============================] - 83s 267ms/step - loss: 0.2121 - sparse_categorical_accuracy: 0.9398 - val_loss: 0.0955 - val_sparse_categorical_accuracy: 0.9759


In [29]:
model.evaluate(test_ds)

78/78 [==============================] - 5s 67ms/step - loss: 1.2411 - sparse_categorical_accuracy: 0.6922


[1.241100788116455, 0.692215085029602]